In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))

# print(f'当前运行在：{NOTEBOOK_PATH} 目录下。')

PATH_FOR_RESULT: Path = NOTEBOOK_PATH.joinpath('RESULT')

if not PATH_FOR_RESULT.exists():
    PATH_FOR_RESULT.mkdir()

# 读取中国金融期货交易所（中金所）的历史数据

解压出来的中金所数据，是 csv 格式的。

读取 csv 文件，使用 Python 的 csv 模块即可。

## 期货日线数据

根据解读 csv 文件的字段。一个期货的基本数据（日线）如下：

- 合约代码
- 日期
- 开盘价
- 最高价
- 最低价
- 收盘价
- 结算价
- 成交量
- 成交额
- 持仓量

交易所还会提供：

- 前结算价
- 持仓量变化
- 涨跌1（今收盘价－前结算价）
- 涨跌2（今结算价－前结算价）


## 期权日线数据

所有字段都和期货一致，交易所提供的部分再增加一个 Delta。


## 需要注意的地方

1. 数据方面

    1. 成交量、持仓量是单边计算还是双边计算，需要找到时间点做切换。

    2. 成交额的单位（万元），是单边计算还是双边计算。

2. 编程方面

    1. 数据类型的表达范围问题。比如整数型（Integer）的最大范围。
    
    2. csv 文件中有些数据是空的，程序需要能够处理。
    
    3. 如何根据合约代码区分期货还是期权？
    
        国内期货品种代码为 1~2 个英文字母，后接四位阿拉伯数字（郑商所是三位，不过我们储存的数据众多，还是补齐四位为妥），总计不超过六个字符。
        
        国内期权代码首先是相应的期货品种，然后是 P 或者 C，再接行权价，中间可能有“-”分隔。因此长度远超六位。

In [2]:
from typing import Dict, List, Tuple, Any
import datetime as dt
import csv

def read_cffex_data(data_file: Path) -> Tuple[List[Dict[str, Any]]]:
    """
    读取中国金融期货交易所（中金所，CFFEX）的历史交易数据 (csv 文件)。
    :param data_file: Path，待读取的文件。
    :return: Tuple，共两项，每一项都是一个列表。
             前一个是期货数据，后一个是期权数据。
             每一个列表都是一个字典，字典的 key 是 str，为 key 的字典。
    """
    result_futures: List[Dict[str, Any]] = []
    result_option: List[Dict[str, Any]] = []
    
    # 从文件名中获得日期。
    filename: str = data_file.name[:8]
    date: dt.date = dt.date(
        year=int(filename[:4]),
        month=int(filename[4:6]),
        day=int(filename[6:8])
    )
    
    # 打开 <data_file> 读取数据。
    with open(data_file, mode='r', encoding='gbk') as csv_file:
        reader = csv.DictReader(csv_file)
        
        # 按行循环读取。
        for row in reader:
            # 忽略 <合约代码> 列为 <小计>、<合计> 的行。
            if row['合约代码'] == '小计' or row['合约代码'] == '合计':
                continue
            
            # 捕捉异常，打印出错的行，以便在出现我们未考虑的情况时，提供调试信息用于修改。
            try:
                # 合约代码，去除两端的空白（空格）
                symbol = row['合约代码'].strip()
                
                # 合约代码长度不超过6，期货
                if len(symbol) <= 6:
                    result_futures.append(
                        {
                            'exchange': 'CFFEX',
                            'date': date,
                            'symbol': symbol,
                            'open': float(row['今开盘']) if len(row['今开盘']) > 0 else 0.0,
                            'high': float(row['最高价']) if len(row['最高价']) > 0 else 0.0,
                            'low': float(row['最低价']) if len(row['最低价']) > 0 else 0.0,
                            'close': float(row['今收盘']) if len(row['今收盘']) > 0 else 0.0,
                            'settlement': float(row['今结算']),
                            'previous_settlement': float(row['前结算']),
                            'volume': int(row['成交量']) if len(row['成交量']) > 0 else 0,
                            'amount': float(row['成交金额']) if len(row['成交金额']) > 0 else 0.0,
                            'open_interest': int(float(row['持仓量'])),
                            'change_on_close': float(row['涨跌1']),
                            'change_on_settlement': float(row['涨跌2']),
                            'change_on_open_interest': int(float(row['持仓变化'])),
                        }
                    )
                # 合约代码长度超过6，期权
                else:
                    result_option.append(
                        {
                            'exchange': 'CFFEX',
                            'date': date,
                            'symbol': symbol,
                            'open': float(row['今开盘']) if len(row['今开盘']) > 0 else 0.0,
                            'high': float(row['最高价']) if len(row['最高价']) > 0 else 0.0,
                            'low': float(row['最低价']) if len(row['最低价']) > 0 else 0.0,
                            'close': float(row['今收盘']) if len(row['今收盘']) > 0 else 0.0,
                            'settlement': float(row['今结算']),
                            'previous_settlement': float(row['前结算']),
                            'volume': int(row['成交量']) if len(row['成交量']) > 0 else 0,
                            'amount': float(row['成交金额']) if len(row['成交金额']) > 0 else 0.0,
                            'open_interest': int(float(row['持仓量'])),
                            'change_on_close': float(row['涨跌1']),
                            'change_on_settlement': float(row['涨跌2']),
                            'change_on_open_interest': int(float(row['持仓变化'])),
                            'delta': float(row['Delta']),
                        }
                    )
            except ValueError as e:
                print(f'读取文件 {csv_file} 时发生错误。发生错误的行内容为：\n\t{row}')
    
    return result_futures, result_option

# 测试
# 用前面第4轮解压的文件，选了一个离现在比较近的日期，内容比较多一些。
test_data_file: Path = PATH_FOR_RESULT.joinpath('unzipped', '20200820_1.csv')

quote_futures, quote_option = read_cffex_data(test_data_file)

print('期货数据：')
for item in quote_futures:
    print(item)

print('-' * 40)

print('期权数据：')
for item in quote_option:
    print(item)

期货数据：
{'exchange': 'CFFEX', 'date': datetime.date(2020, 8, 20), 'symbol': 'IC2008', 'open': 6600.0, 'high': 6663.4, 'low': 6564.6, 'close': 6587.2, 'settlement': 6589.0, 'previous_settlement': 6662.0, 'volume': 51483, 'amount': 6808092.2, 'open_interest': 28408, 'change_on_close': -74.8, 'change_on_settlement': -73.0, 'change_on_open_interest': -26567}
{'exchange': 'CFFEX', 'date': datetime.date(2020, 8, 20), 'symbol': 'IC2009', 'open': 6488.0, 'high': 6569.6, 'low': 6460.2, 'close': 6495.8, 'settlement': 6489.8, 'previous_settlement': 6560.4, 'volume': 94511, 'amount': 12303295.716, 'open_interest': 119772, 'change_on_close': -64.6, 'change_on_settlement': -70.6, 'change_on_open_interest': 26458}
{'exchange': 'CFFEX', 'date': datetime.date(2020, 8, 20), 'symbol': 'IC2012', 'open': 6287.8, 'high': 6351.4, 'low': 6250.0, 'close': 6276.4, 'settlement': 6273.8, 'previous_settlement': 6348.8, 'volume': 16775, 'amount': 2112116.892, 'open_interest': 52079, 'change_on_close': -72.4, 'change_

下一步，我们着手把数据[写入数据库](save_to_database.ipynb)。